# AES - Symmetric Cryptography and the Rijndael Cryptographic System

## MATH 157 - Final Project

**Tomás Gillanders**  
**PID: A17694974**

In [38]:
using Pkg
# PKG.add("LinearAlgebra")
# Pkg.add("Nemo")
# Pkg.add("Primes")
# Pkg.add("SymPy")

# Import the Libraries used by the project.
using LinearAlgebra;
using Nemo;
using Primes;
# using SymPy;

## Introduction

...

## Part 1: Finite Fields - Theory

### §1.1 Laying the Foundations

***Definition 1.1.1*** **(Abelian Group)**: Let $G$ be a non-empty set on which there is defined a binary operation $+: G \times G \rightarrow G$.  We say that $(G,+)$ is a group iff the following axioms hold:
- Closure:           $\qquad \qquad \forall a,b \in G, \; a+b \in G$
- Associativity:     $\qquad \ \  \forall a,b,c \in G, \; (a+b)+c = a+(b+c)$
- Identity Element:  $\quad \  \exists e \in G, \; \forall a \in G, \; a + e = e + a = a$
- Inverses:          $\qquad \quad \ \ \ \ \forall a \in G, \; \exists (-a) \in G, \; a + (-a) = (-a) + a = e$
- Commutivity:       $\qquad \ \ \forall a,b \in G, \; a+b = b+a$

For an abelian group, the operation, $+$, is often reffered to as addition, and the identity element is denoted by $\textbf{0}$.

***Definition 1.1.2*** **(Ring)**: Let $R$ be a non-empty set on which there is defined two binary operations, denoted $+,\bullet: R \times R \rightarrow R$.  We say that $(R, +, \bullet \,)$ is a ring iff:
1. The algebraic structure $(R, +)$ constitutes an abelian group.
2. The operation, $\bullet$, is closed, associative and there exists an identity element (i.e. $(R,\bullet \,)$ constitutes a monoid).
3. Distributivity:   $ \qquad \ \ \  \forall a,b,c \in R: \; a \bullet(b+c) = a \bullet b + a \bullet c $

We say that $(R,+,\bullet \,)$ is a commutative ring if the operation, $\bullet$, is commutative.  The identity element for $\cdot$ is often denoted by $\textbf{1}$ (while the identity element for $+$ is denoted by $\textbf{0}$, as above).

***Definition 1.1.3*** **(Field)**: Let $F$ be a non-empty set on which there is defined two binary operations $+,\bullet: F \times F \rightarrow F$.  We say that $(F, +, \bullet)$ is a field iff:
1. $(F, +, \bullet)$ is a commutative ring.
2. $\forall a \in F \setminus \{\textbf{0}\}, \; \exists a^{-1} \in F \setminus \{\textbf{0}\}$ such that $a \bullet a^{-1} = a^{-1} \bullet a = \textbf{1}$.


---
---

### §1.2 Finite Fields

In the implementation of AES/Rijndael, the theory of Finite (Galois) Fields plays a central role in the encryption and decyrption processes.


***Definition 1.2.1*** **(Finite Field)**: A Finite Field is a field with a finite number of elements.  The number of elements in the set is called the ***order*** of the field.

>*Note: When discussing fields for the remainder of this project, it is implicite that the fields in question are finite, unless otherwise stated.*

***Theorem 1.2.2*** *(Existence of Finite Fields)*: For every prime $p$ and positive integer $m$, there exists a finite field of order $p^m$.  We say that $p$ is the ***characteristic*** of the field.

***Theorem 1.2.3***: All finite fields of the same order are *isomorphic*. [isomorphic reference].

*Theorem 1.2.3* tells us that that while (finite) fields of the same order may differ in the representation of their elements, allgebraically their structures are identical.  Therefore, combining *Theorems 1.2.2* and *1.2.3*, we have that for every prime power, there exists exactly one finite.  We denote the Finite Field/Galois Field of order $p^n$ by:

$$ GF(p^n) $$

---
---





### §1.3 Operations on Finite Fields & Polynomial Representations

#### Fields of Prime Order:

Consider the field $GF(p)$, where $p$ is a prime. The elements of $GF(p)$ can be represented by the integers 

$$ 0, 1, 2, ..., p-2, p-2 $$ 

Then the two operations of field addition and multiplication on $GF(p)$ are the familiar operations of integer addition modulo $p$ and integer multiplication modulo $p$. 

---

### Paricipation Check

Consider the field $GF(17)$.  Using the `Nemo` library, or otherwise, find $a,b,c \in GF(17)$, where
1. $a = 15 + 5$
2. $b = 13 \bullet 3$
3. $10 = 13 \bullet c$

In [9]:
# YOUR ANSWER HERE
# ...

# Answers:
F17 = GF(17)

@show a = F17(15) + 5
@show b = F17(13) * 3
@show c = F17(10) // 13;  # or use `powermod()` without Nemo

a = F17(15) + 5 = 3
b = F17(13) * 3 = 5
c = F17(10) // 13 = 6


---

#### Fields of Non-Prime Order:

Consider the field $GF(p^n)$, with $n>1$.  For such fields the operations of addition and multiplication cannot be defined/represented in terms of integer addation and multiplication as was done above.

> **Example**: Consider $GF(4)$.  *Theorem 1.2.2* tells us that $GF(4)$ is a field (with the appropriate field operations), since $4 = 2^2$.  However, if we consider the element $2 \in GF(4)$, we note that 
>
>$$ \forall n \in \mathbb{Z}: \quad 2 \bullet n \equiv 0 \pmod{4} \quad \text{or} \quad 2 \bullet n \equiv 2 \pmod{4} $$
>
>Thus, there does not exist $2^{-1} \in \mathbb{Z}$ such that $2 * 2^{-1} \equiv 1 \pmod{4}$.  This means that integer multiplication modulo 4 cannot be one of the operations defined on $GF(4)$.



In [18]:
# Check for the first 1001 non-negative integers
ints = collect(0:1000)

1 ∈ (2 .* ints) .% 4

false

In order to define our field operations on $GF(p^n)$, $n>1$, we move to representing its elements in terms of polynomials, rather than the integers $0,1,...,p-1$.

***Definition 1.3.1*** **(Polynomial over a Field)**:  Let $F$ be a field.  A polynomial over $F$ is an expression of the form:

$$ p(x) = \sum_{i = 0}^{n-1} a_i x^i $$

where $x$ is called the ***indeterminate*** of the polynomial and $a_i \in F$ are the ***coefficients***.

***Definition 1.3.2*** **(Degree of Polynomial)**: The ***degree*** of a polynomial is the least $l$ such that $a_j = 0, \; \forall j > l$.  

We denote by $F[x]$ the *ring* of all polynomials over $F$, and by $F[x]|_l$ the *set* of all polynomials over $F$ with degree strictly less than $l$.

---

Using *Definition 1.3.1* we can now represent the elements of $GF(p^n)$ as follows.  Consider an integer $b$ in the range $0,1,...,p^n-1$:

- First write $b$ with respect to the base $p$.  This gives us a list of terms:

    $$ (b)_p = (b_{n-1} b_{n-2} ... b_2 b_1 b_0)_p $$
    where
    $$ b = b_{n-1} \cdot p^{n-1} + ... + b_1 \cdot p^1 + b_0 \cdot p^0 $$ 
    
    
- Then, since each $b_i$ is in the range $0,1,...,p-1$, we can identify $b$ with a unique polynomial in $GF(p)|_n$ as follows:

$$ b \longleftrightarrow b(x) =  b_{n-1} x^{n-1} + ... +  b_2 x^2 + b_1 x + b_0$$ 

---

We can now define our operations on $GF(p^n)$ as follows:

Consider two elements $a,b \in GF(p^n)$.  We identify $a,b$ with polynomials in the field $GF(p)[x]|_n$:

$$ a \longleftrightarrow a(x) \qquad \text{and} \qquad b \longleftrightarrow b(x) $$

We recall that by *Theorem 1.2.3*, $GF(p^n)$ and $GF(p)[x]|_n$ are algebraicly the same.* [see appendix on discussion of $GF(p)[x]|_n$ as a field.] 

- **Addition**: $a + b \longleftrightarrow a(x) + b(x)$ where the addition of the coefficients of like powered terms in the polynomial sum occurs in $GF(p)$.
- **Multiplication**: We note that for $GF(p)[x]|_n$ to be closed under polynomial multiplication an irreducible ***reduction polynomial*** of degree $n$, $m(x)$, must be chosen.  Multiplication is then defined in terms of:

$$ a \bullet b \longleftrightarrow a(x) \bullet b(x) \pmod{m(x)} $$

---

\* see appendix/link/book... on discussion of $GF(p)[x]|_n$ as a field.

---

### §1.4 Example: Applying Finite Fields and AES

In the implementation of AES, bits of data are grouped into bytes.  Therefore, the set of all possible byte-values can be thought of as elements (polynomials) of the field $GF(2)[x]|_8$ or, as discussed above, $GF(2^8)$.

Thus, when looking to encrypt a given plaintext byte, the operations and theory of finite fields can be applied.

In the specification of AES, bytes are considered as polynomials , with 'addition' and 'multiplication' of bytes as discussed above.  We note also that the following irreducible polynomial is used as reduction polynomial for all multiplication operations:

$$ m(x) = x^8 + x^4 + x^3 + x + 1 $$

Note that while the byte multiplication is used in the AES specification, no computation of byte multiplication is required in its implmentation.  This is further discussed in §2.(...).


> **Byte Addition and its Implemenation**:
>
> Consider the elements $53, 12 \in GF(2^8)$.  We wish to find the sum $53 + 12 \in GF(2^8)$. We carry out this addition as follows:


In [90]:
# Using the Nemo library intialise the Field GF(2) and the Polynomial Ring GF(2)[x]
@show GF2        = GF(2)
@show R_GF2_x, x = GF2["x"];

GF2 = GF(2) = Galois field with characteristic 2
(R_GF2_x, x) = GF2["x"] = (Univariate Polynomial Ring in x over Galois field with characteristic 2, x)


In [91]:
# Find the binary representations of 53 and 12.
@show bin53 = string(53, base = 2)
@show bin12 = string(12, base = 2);

bin53 = string(53, base = 2) = "110101"
bin12 = string(12, base = 2) = "1100"


In [92]:
# Parse and return as vectors.
@show vec53 = parse.(Int, split(bin53, ""))
@show vec12 = parse.(Int, split(bin12, ""));

vec53 = parse.(Int, split(bin53, "")) = [1, 1, 0, 1, 0, 1]
vec12 = parse.(Int, split(bin12, "")) = [1, 1, 0, 0]


In [93]:
# Produce the polynomial representations
@show pol53 = R_GF2_x(reverse(vec53))
@show pol12 = R_GF2_x(reverse(vec12));

pol53 = R_GF2_x(reverse(vec53)) = x^5 + x^4 + x^2 + 1
pol12 = R_GF2_x(reverse(vec12)) = x^3 + x^2


In [95]:
# Add the polynomials over GF(2)[x]
@show pol53 + pol12;

pol53 + pol12 = x^5 + x^4 + x^3 + 1


> Thus, we have that the addition of 53 and 12 over $GF(2^8)$ corresponds to the number with binary representation
> $$ 111001 $$
> We use the `parse()` function again to parse this number in decimal.

In [98]:
@show answer = parse(Int, "111001" , base = 2);

answer = parse(Int, "111001", base = 2) = 57


> Thus, we have that in $GF(2^8)$, $53 + 12 = 57$.
>
> However, on closer examination of the above method, we note that the operation in fact simply implemented, without consideration of polynomial addition. 

---

### Participation Check

Compute $157 + 200$ in $GF(2^8)$, without using the polynomial addition method above. (This can be accomplished in one short line of code).

In [100]:
# Your Answer Here



**Answer**: Looking at the above example, we notice that addition of elements of $GF(2^8)$ is simply the operation of taking the ***Bitwise XOR*** of the two elements.  Julia allows us to apply this operation using the $\veebar$ (\xor - tab) symbol.

Note that often the symbol $\oplus$ is used to represent XOR in literature.  However, here we use $\veebar$ to remain consistent with the Julia syntax.

In [102]:
@show 157 ⊻ 200;

157 ⊻ 200 = 85


---

---
---
---

## Part 2: The Encryption Algorithm

### §2.1 Overview

AES is a ***Key-Iterated Block Cipher*** that acts on a square $4 \times 4$ array of bytes (this is how the plaintext data is grouped).

***Definition 2.1.1*** **(Block Cipher)**: A *Block Cipher* is a function that operates on a *Plaintext Block* of fixed length (number of bits), and returns a *ciphertext block* of the same length, under the influence of a cipher key, $k$.

An ***iterative block cipher*** is a block cipher in which some function(s) (called ***Boolean Permutations***) are repeatedly applied to the given ***state*** of the block of data. These Boolean Permutations are called the ***Round Transformations***, and every application of a Round Transformation is called a ***Round***.

Each Round is dependent on a ***Round Key***, which is generated using the original cipher key (discussed later).  If we call $k^{(i)}$ the $i^{th}$ round key, with $k^{(0)}$ the original cipher key, then the concatenation of all round keys is called the ***Expanded Key***, denoted $K$:

$$ K = k^{(0)}|k^{(2)}|k^{(3)}|...|k^{(r)} $$

where $r$ is the number of round applications.

If we take $\rho$ to be the round transformation, $\sigma[k^{(i)}]$ to be the byte addition (bitwise XOR) of the current *state* with the $i^{th}$ round key and $B[k]$ to be the block cipher, then AES takes the form:

$$ B[k] = \sigma[k^{(r)}] \circ \rho' \circ \sigma[k^{(r-1)}] \circ \rho \circ \cdots \sigma[k^{(1)}] \circ \rho \circ \sigma[k^{(0)}] $$

where $\rho'$ is the final round, which differs slightly.

We now discuss the structure of each round.

### §2.2 The Round Tranformation, `Round`

In discussing the round transformation of the AES Encryption Algorithm, we follow the naming conventions as layed out in Daemen and Rijmen's "The Design of Rijndael; AES - The Advanced Encryption Standard". [insert reference]



#### §2.2.1 `SubBytes`